In [1]:
# Task1 Establish a connection pool
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

In [2]:
dbconfig={"database":"little_lemon_db", "user":"ryan", "password":"5555"}

In [7]:
try:
    pool = MySQLConnectionPool(pool_name = "pool_a",
                           pool_size = 2, 
                           **dbconfig)
    print("The connection pool is created with a name: ",pool.pool_name)
    print("The pool size is:",pool.pool_size)
except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)

The connection pool is created with a name:  pool_a
The pool size is: 2


In [ ]:
# Task2 Adding three new guests to bookings table

In [24]:

try:
        guest1_connected = pool.get_connection()
        print("Guest1 is connected.\n")
        cursor = guest1_connected.cursor()
        insert_bookings="""
        INSERT INTO Bookings (BookingID, TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
        VALUES
        (7, 8, 'Anees','Java','18:00:00',6);"""
        cursor.execute(insert_bookings)
        guest1_connected.commit()
        print("Booking of Guest1 is finished.\n")
except:
        print("No more connections are available.")
        print("Adding new connection in the pool.")

Guest1 is connected.

Booking of Guest1 is finished.



In [29]:
try:
        guest2_connected = pool.get_connection()
        print("Guest2 is connected.\n")
        cursor = guest2_connected.cursor()
        insert_bookings="""
        INSERT INTO Bookings (BookingID, TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
        VALUES
        (8, 5, 'Bald','Vin','19:00:00',6);"""
        cursor.execute(insert_bookings)
        guest2_connected.commit()
        print("Booking of Guest2 is finished.\n")
except:
        print("No more connections are available.")
        print("Adding new connection in the pool.")

Guest2 is connected.

Booking of Guest2 is finished.



In [32]:
try:
        guest3_connected = pool.get_connection()
        print("Guest3 is connected.\n")
        cursor = guest3_connected.cursor()
        insert_bookings="""
        INSERT INTO Bookings (BookingID, TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
        VALUES
        (9, 12, 'Jay','Con','19:30:00',6);"""
        cursor.execute(insert_bookings)
        guest3_connected.commit()
        print("Booking of Guest3 is finished.\n")
except:
        print("No more connections are available.")
        print("Adding new connection in the pool.")
         # Create a connection
        connection=connector.connect(user="ryan",password="5555")
        # Add the connection into the pool
        pool.add_connection(cnx=connection)
        print("A new connection is added in the pool.\n")
        
        guest3_connected = pool.get_connection()
        print("Guest3 is connected.\n")
        cursor = guest3_connected.cursor()
        insert_bookings="""
        INSERT INTO Bookings (BookingID, TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
        VALUES
        (9, 12, 'Jay','Con','19:30:00',6);"""
        cursor.execute(insert_bookings)
        guest3_connected.commit()
        print("Booking of Guest3 is finished.\n")


No more connections are available.
Adding new connection in the pool.
A new connection is added in the pool.

Guest3 is connected.

Booking of Guest3 is finished.



In [ ]:
guest1_connected.close()
guest2_connected.close()
guest3_connected.close()

In [37]:
# Task3-1 The name and EmployeeID of the Little Lemon manager
connection = connector.connect(user="ryan",password="5555")
cursor = connection.cursor()
cursor.execute("USE little_lemon_db")

In [41]:
query_manager = """
SELECT EmployeeID AS Manager_ID,
Name AS Manager_Name
FROM employees
WHERE Role = 'Manager';
"""
cursor.execute(query_manager)
results = cursor.fetchall()
columns = cursor.column_names
print(columns)
for result in results:
    print(result)

('Manager_ID', 'Manager_Name')
(1, 'Mario Gollini')


In [46]:
# Task3-2 employee with highest salary
query_salary = """
SELECT 
Name AS Employee_Name,
Role AS Employee_Role
FROM employees
ORDER BY Annual_Salary DESC LIMIT 1;
"""
cursor.execute(query_salary)
results = cursor.fetchall()
print("The Name and Role of the employee with the highest salary is:")
for result in results:
    print(result)

The Name and Role of the employee with the highest salary is:
('Mario Gollini', 'Manager')


In [49]:
# Task3-3 number of guests booked between 18:00 and 20:00
query_guest_arrival = """
SELECT 
COUNT(BookingID)
FROM Bookings
WHERE HOUR(BookingSlot) IN (18,20);
"""
cursor.execute(query_guest_arrival)
results = cursor.fetchall()
print("The number of guests booked between 18:00 and 20:00:")
for result in results:
    print(result)

The number of guests booked between 18:00 and 20:00:
(3,)


In [62]:
# Task3-4 guests waiting to be seated
query_guest_waiting = """
SELECT
CONCAT(Bookings.GuestFirstName, ' ', Bookings.GuestLastName) AS Guest_Name,
BookingID AS ID
FROM Bookings
WHERE EmployeeID = 6
ORDER BY BookingSlot;
"""
cursor.execute(query_guest_waiting)
results = cursor.fetchall()
columns = cursor.column_names
print(columns)
for result in results:
    print(result)

('Guest_Name', 'ID')
('Anees Java', 7)
('Bald Vin', 8)
('Jay Con', 9)


In [51]:
# Task4 Create stored procedure: BasicSalesReport. 
cursor.execute("DROP PROCEDURE IF EXISTS BasicSalesReport;")

In [52]:
stored_procedure_query="""
CREATE PROCEDURE BasicSalesReport()
BEGIN

SELECT 
SUM(BillAmount) AS Total_Sale,
AVG(BillAmount) AS Average_Sale,
MIN(BillAmount) AS Min_Bill_Paid,
MAX(BillAmount) AS Max_Bill_Paid
FROM Orders;

END
"""

In [53]:
cursor.execute(stored_procedure_query)

In [54]:
cursor.callproc("BasicSalesReport")

()

In [55]:
results = next(cursor.stored_results())
results = results.fetchall()

In [56]:
for column_id in cursor.stored_results():
    cols = [column[0] for column in column_id.description]

In [60]:
print("Basic sales report:")
for result in results:
    print(cols[0],':',result[0])
    print(cols[1],':',result[1])
    print(cols[2],':',result[2])
    print(cols[3],':',result[3])
 

Basic sales report:
Total_Sale : 243
Average_Sale : 48.6000
Min_Bill_Paid : 37
Max_Bill_Paid : 86


In [61]:
#Task5; 
connection = pool.get_connection()
cursor=connection.cursor(buffered=True)

In [63]:
query_upcomig_booking="""
SELECT 
Bookings.BookingSlot,
CONCAT(Bookings.GuestFirstName," ",Bookings.GuestLastName) AS Guest_Name,
Employees.Name AS Employee_Name,
Employees.Role AS Employee_Role
FROM Bookings 
INNER JOIN 
Employees ON Bookings.EmployeeID=Employees.EmployeeID
ORDER BY Bookings.BookingSlot ASC;
"""

In [64]:
cursor.execute(query_upcomig_booking)
results=cursor.fetchmany(size=3)

In [67]:
for result in results:
    print("BookingSlot",result[0])
    print("Guest_name:",result[1])
    print("Assigned to:",result[2],"[{}]".format(result[3]))


BookingSlot 15:00:00
Guest_name: Vanessa McCarthy
Assigned to: Giorgos Dioudis [Head Chef]
BookingSlot 17:30:00
Guest_name: Marcos Romero
Assigned to: Fatma Kaya [Assistant Chef]
BookingSlot 18:00:00
Guest_name: Anees Java
Assigned to: John Millar [Receptionist]


In [ ]:
    
connection.close()